In [2]:
!npm install -g localtunnel
!pip install flask-cors

import zipfile
import os
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
import json
import threading
import time

app = Flask(__name__)
CORS(app)  # Permitir solicitudes CORS
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'OPTIONS'])
def home():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    return "Server is running", 200

@app.route('/upload', methods=['POST', 'OPTIONS'])
def upload_file():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    if 'file' not in request.files:
        return 'No file part', 400
    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400
    if file:
        filename = file.filename
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        return 'File uploaded successfully', 200

def build_cors_preflight_response():
    response = jsonify({"message": "CORS preflight"})
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "GET, POST, OPTIONS, PUT, DELETE")
    return response

def process_uploaded_file(zip_path):
    extract_dir = '/content/extracted'
    os.makedirs(extract_dir, exist_ok=True)

    # Extraer archivo zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # Listar archivos extraídos
    extracted_files = os.listdir(extract_dir)
    print(extracted_files)

    # Aquí puedes agregar el código para procesar los archivos extraídos
    # ...

# Comprobar si hay archivos en el directorio de subidas y procesarlos
uploaded_files = os.listdir(UPLOAD_FOLDER)
for file in uploaded_files:
    if file.endswith('.zip'):
        process_uploaded_file(os.path.join(UPLOAD_FOLDER, file))

@app.route('/web/<path:path>')
def serve_file(path):
    return send_from_directory('/content/web', path)

@app.route('/modify', methods=['POST', 'OPTIONS'])
def modify_config():
    if request.method == 'OPTIONS':
        return build_cors_preflight_response()
    new_value = request.json.get('ntxCafCompressed')
    with open('/content/web/config.json', 'r') as config_file:
        config = json.load(config_file)

    config['ntxCafCompressed'] = new_value

    with open('/content/web/config.json', 'w') as config_file:
        json.dump(config, config_file)

    return jsonify({"status": "success"})

@app.route('/check_auth', methods=['GET'])
def check_auth():
    return jsonify({"status": "authenticated"}), 200

def run_app():
    app.run(host='0.0.0.0', port=5000)

thread = threading.Thread(target=run_app)
thread.start()

time.sleep(5)  # Espera a que localtunnel se inicie

# Obtener la contraseña del túnel
tunnel_password = os.popen('curl https://loca.lt/mytunnelpassword').read().strip()
print(f"Tunnel password: {tunnel_password}")

!lt --port 5000
